Setup rclone and restic


In [ ]:
%env RESTIC_REPOSITORY=rclone:dest:colab
%env RESTIC_PASSWORD=CHANGEME
%env RESTIC_HOST=google-colab
%env DEST_PATH=local:/content/dest

!mkdir -p /root/.config/rclone/
!mkdir -p "/content/source/"
!mkdir -p "/content/data/"
!mv /content/sample_data /content/data/sample_data
!mkdir -p /content/dest

!curl https://rclone.org/install.sh | sudo bash

!curl -L https://github.com/restic/restic/releases/download/v0.12.0/restic_0.12.0_linux_amd64.bz2 | bunzip2 > /usr/bin/restic
!chmod +x /usr/bin/restic

!rclone --version
!restic version

Generate rclone.conf

By default, this script looks for source and dest.

Set up these 2 configs for the script to work

In [ ]:
%%shell

cat > /root/.config/rclone/rclone.conf <<CONFIG
[local]
type = local

[source]
type = alias
remote = local:/content/data

[dest]
type = alias
remote = ${DEST_PATH}
CONFIG

cat /root/.config/rclone/rclone.conf

Init the repository if required

In [ ]:
!restic init

Mount the source directory if required

In [ ]:
%%shell
mounts=(\
    "sample_data"\
)

backups=(\
    "sample_data"\
)


# Iterate the string array using for loop
for mount in "${mounts[@]}"; do
    mkdir -p  "/content/source/$mount"
    rclone mount "source:/$mount" "/content/source/$mount" --daemon
    sleep 5
done

sleep 5

for backup in "${backups[@]}"; do
    restic backup --verbose --host "$RESTIC_HOST" "/content/source/$backup"
done

# Iterate the string array using for loop
for val in "${mounts[@]}"; do
    umount "/content/source/$val"
done

# keep the most recent snapshot per path
restic forget --host "$RESTIC_HOST" -l 1
restic snapshots

In [ ]:
!restic unlock --remove-all
!restic forget --host "$RESTIC_HOST" -l 1
!restic snapshots